# Feedforward classification of the NMIST data
### Advanced Deep Learning 2024
This notebook was written originally Jon Sporring (mailto:sporring@di.ku.dk) and heavily inspired by https://clay-atlas.com/us/blog/2021/04/22/pytorch-en-tutorial-4-train-a-model-to-classify-mnist.

We consider the Modified National Institute of Standards and Technology database of handwritten digits (MNIST): http://yann.lecun.com/exdb/mnist/

## Installs

On non-colab system, is usually good to make an environment and install necessary tools there. E.g., anaconda->jupyter->terminal create an environment, if you have not already, and activate it:
```
conda create -n adl python=3.9
conda activate adl
```
then install missing packages such as:
```
conda install ipykernel torch matplotlib torchmetrics scikit-image jpeg
conda install -c conda-forge segmentation-models-pytorch ipywidgets
```
and if you want to add it to jupyter's drop-down menu
```
ipython kernel install --user --name=adl
```
Now reload the jupyter-notebook's homepage and make a new or load an existing file. On colab, the tools have to be installed everytime

In [28]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if IN_COLAB:
    !pip3 install torch matplotlib torchmetrics scikit-image segmentation-models-pytorch

## Imports

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as dset
from torchvision import datasets, transforms

## Set global device

In [30]:
# GPU
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('GPU State:', device)

GPU State: cpu


In [31]:
# # Transformation and DataLoader
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.5,), (0.5,))
# ])
# trainSet = datasets.MNIST(root='MNIST', download=True, train=True, transform=transform)
# testSet = datasets.MNIST(root='MNIST', download=True, train=False, transform=transform)
# trainLoader = dset.DataLoader(trainSet, batch_size=64, shuffle=True)
# testLoader = dset.DataLoader(testSet, batch_size=64, shuffle=False)

# # CNN Model Definition
# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.main = nn.Sequential(
#             nn.Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1)),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1)),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Flatten(),
#             nn.Linear(800, 10),
#             nn.LogSoftmax(dim=1)
#         )

#     def forward(self, input):
#         return self.main(input)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# net = CNN().to(device)

# # Training Parameters
# epochs = 4
# lr = 0.002
# loss_fn = nn.NLLLoss()
# optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)

# # Training Loop
# def training_loop(model, loss_fn, optimizer, data_loader, epochs):
#     for epoch in range(epochs):
#         model.train()
#         for images, labels in data_loader:
#             images, labels = images.to(device), labels.to(device)
#             outputs = model(images)
#             loss = loss_fn(outputs, labels)
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#         print(f'Epoch {epoch+1}/{epochs} completed')

# # Evaluation Function
# def evaluate_model(model, data_loader):
#     model.eval()
#     total = 0
#     correct = 0
#     class_correct = list(0. for i in range(10))
#     class_total = list(0. for i in range(10))
#     with torch.no_grad():
#         for images, labels in data_loader:
#             images, labels = images.to(device), labels.to(device)
#             outputs = model(images)
#             _, predicted = torch.max(outputs, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()
#             c = (predicted == labels).squeeze()
#             for i in range(labels.size(0)):
#                 label = labels[i]
#                 class_correct[label] += c[i].item()
#                 class_total[label] += 1
#     return correct, total, class_correct, class_total

# # Train and Evaluate
# print('Training on %d images' % len(trainSet))
# training_loop(net, loss_fn, optimizer, trainLoader, epochs)
# print('Training Finished.\n')

# # Test the model
# correct, total, class_correct, class_total = evaluate_model(net, testLoader)
# print('Accuracy of the network on the %d test images: %d %%' % (len(testSet), (100 * correct / total)))
# for i in range(10):
#     print('Accuracy of %d: %3f' % (i, (class_correct[i] / class_total[i] if class_total[i] != 0 else 0)))

# # Save the notebook content if needed


## Functions

In [32]:
def training_loop(model, loss, optimizer, loader, epochs, verbose=True, device=device):
    """
    Run training of a model given a loss function, optimizer and a set of training and validation data.
    """

    # Train
    for epoch in range(epochs):
        running_loss = 0.0

        for times, data in enumerate(loader):
            inputs, labels = data[0].to(device), data[1].to(device)
            inputs = inputs.view(inputs.shape[0], -1)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Foward + backward + optimize
            outputs = model(inputs)
            loss_tensor = loss(outputs, labels)
            loss_tensor.backward()
            optimizer.step()

            # Print statistics
            running_loss += loss_tensor.item()
            if verbose:
                if times % 100 == 99 or times+1 == len(loader):
                    print('[%d/%d, %d/%d] loss: %.3f' % (epoch+1, epochs, times+1, len(loader), running_loss/2000))

In [33]:
def evaluate_model(model, loader, device=device):
    """
    Evaluate a model 'model' on all batches of a torch DataLoader 'data_loader'.

    Returns: the total number of correct classifications,
             the total number of images
             the list of the per class correct classification,
             the list of the per class total number of images.
    """

    # Test
    correct = 0
    total = 0

    with torch.no_grad():
        for data in loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = inputs.view(inputs.shape[0], -1)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    class_correct = [0 for i in range(10)]
    class_total = [0 for i in range(10)]

    with torch.no_grad():
        for data in loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            inputs = inputs.view(inputs.shape[0], -1)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(10):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    return (correct, total, class_correct, class_total)


## Main program

In [34]:
# Transform
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,)),]
)

In [35]:
# Data
trainSet = datasets.MNIST(root='MNIST', download=True, train=True, transform=transform)
testSet = datasets.MNIST(root='MNIST', download=True, train=False, transform=transform)
trainLoader = dset.DataLoader(trainSet, batch_size=64, shuffle=True)
testLoader = dset.DataLoader(testSet, batch_size=64, shuffle=False)

In [36]:
# Model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(800, 10),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, input):
        input = input.view(-1, 1, 28, 28)
        return self.main(input)

net = CNN().to(device)
print(net)

CNN(
  (main): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=800, out_features=10, bias=True)
    (8): LogSoftmax(dim=1)
  )
)


In [37]:
# Parameters
epochs = 4
lr = 0.002
loss = nn.NLLLoss()
optimizer = optim.SGD(net.parameters(), lr=0.002, momentum=0.9)

# Train
print('Training on %d images' % trainSet.data.shape[0])
training_loop(net, loss, optimizer, trainLoader, epochs)
print('Training Finished.\n')

# Test
correct, total, class_correct, class_total = evaluate_model(net, testLoader)
print('Accuracy of the network on the %d test images: %d %%' % (testSet.data.shape[0], (100*correct / total)))
for i in range(10):
    print('Accuracy of %d: %3f' % (i, (class_correct[i]/class_total[i])))

Training on 60000 images
[1/4, 100/938] loss: 0.100
[1/4, 200/938] loss: 0.135
[1/4, 300/938] loss: 0.153
[1/4, 400/938] loss: 0.169
[1/4, 500/938] loss: 0.182
[1/4, 600/938] loss: 0.194
[1/4, 700/938] loss: 0.204
[1/4, 800/938] loss: 0.214
[1/4, 900/938] loss: 0.222
[1/4, 938/938] loss: 0.225
[2/4, 100/938] loss: 0.008
[2/4, 200/938] loss: 0.016
[2/4, 300/938] loss: 0.022
[2/4, 400/938] loss: 0.028
[2/4, 500/938] loss: 0.035
[2/4, 600/938] loss: 0.041
[2/4, 700/938] loss: 0.047
[2/4, 800/938] loss: 0.053
[2/4, 900/938] loss: 0.059
[2/4, 938/938] loss: 0.061
[3/4, 100/938] loss: 0.006
[3/4, 200/938] loss: 0.011
[3/4, 300/938] loss: 0.016
[3/4, 400/938] loss: 0.020
[3/4, 500/938] loss: 0.025
[3/4, 600/938] loss: 0.029
[3/4, 700/938] loss: 0.034
[3/4, 800/938] loss: 0.038
[3/4, 900/938] loss: 0.043
[3/4, 938/938] loss: 0.045
[4/4, 100/938] loss: 0.004
[4/4, 200/938] loss: 0.009
[4/4, 300/938] loss: 0.013
[4/4, 400/938] loss: 0.016
[4/4, 500/938] loss: 0.020
[4/4, 600/938] loss: 0.024
[4/